# Create Azure Machine Learning Models

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a model from a local file
- Create a model from an mlflow model
- Create a model from cloud path
- Create a model from a run

**Motivations** - This notebook explains how to register a model in the Azure Machine Learning workspace.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
!pip install azure-ml==0.0.139 --extra-index-url  https://azuremlsdktestpypi.azureedge.net/sdk-cli-v2

In [1]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Model
from azure.identity import InteractiveBrowserCredential
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [3]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 2. Create a model
Azure ML models consist of the binary file(s) that represent a machine learning model and any corresponding metadata. Models can be created from a local file or directory. The created model will be tracked in the workspace under the specified name and version.

The `Model` class can be used to create a model. It accepts the following key parameters:
- `name` - Name of the model.
- `version` - Version of the model. If omitted, Azure ML will autogenerate a version.
- `local_path` - Local path to the model file(s). This can point to either a file or a directory.
- `model_uri` - URI of the model file(s). This can point to either a file or directory.
- `model_format` - Storage format of the model. Applicable for no-code deployment scenarios. Allowed values are `custom`, `mlflow`, `triton`, `openai`
- `description` - Description of the model.

## 2.1 Create a model from a local file
In this sample we will create a model from a local `pkl` file and specify the format of the model to be `custom`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
file_model = Model(
    local_path="mlflow-model/model.pkl",
    model_format="custom",
    name="local-file-example",
    description="v1 Model created from local file."
)
ml_client.models.create_or_update(file_model)

## 2.2 Create a model from an mlflow model
In this sample we will create a model from a local folder. We will also specify the format of the model to be `mlflow`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [30]:
mlflow_model = Model(
    local_path="mlflow-model",
    model_format="mlflow",
    name="local-mlflow-example",
    description="local-mlflow-example"
)
ml_client.create_or_update(mlflow_model)

Creating or updating model
Uploading mlflow-model:   0%|          | 0.00/866 [00:00<?, ?B/s]


Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'local-mlflow-example', 'description': 'local-mlflow-example', 'tags': {}, 'properties': {}, 'id': '/subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/providers/Microsoft.MachineLearningServices/workspaces/mlflowworkspace/models/local-mlflow-example/versions/1', 'base_path': './', 'creation_context': <azure.ml._restclient.v2021_10_01.models._models_py3.SystemData object at 0x7f49251b6be0>, 'serialize': <msrest.serialization.Serializer object at 0x7f49263aa198>, 'version': '1', 'local_path': None, 'model_uri': 'azureml://subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/workspaces/mlflowworkspace/datastores/workspaceblobstore/paths/LocalUpload/4570cbe51af4c414c3f54112f31c7bdf/mlflow-model', 'utc_time_created': None, 'flavors': {}, 'arm_type': 'model_version', 'model_format': 'MLFlow'})

## 2.3 Create a model from cloud path
In this sample we will create a model from a cloud path. We will also specify the format of the model to be `mlflow`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
cloud_model = Model(
    model_uri="azureml://subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/workspaces/mlflowworkspace/datastores/workspaceblobstore/paths/model.pkl",
    #Users could also use,"azureml://datastores/workspaceblobstore/paths/model.pkl" as a shorthand to the same location
    name="cloud-path-example",
    model_format="custom",
    description="Model created from cloud path."
)
ml_client.models.create_or_update(cloud_model)

## 2.4 Create a model from a run
In this sample we will create a model from a run. We will also specify the format of the model to be `mlflow`. The model is initialized with the required parameters. It is then created in the Azure Machine Learning workspace using the `MLClient`.

In [ ]:
run_model = Model(
    model_uri="azureml://subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/workspaces/mlflowworkspace/jobs/7da56f9f-51e4-4a13-8039-55127154f8d3/outputs/artifacts/paths/model/",
    #Users could also use, "azureml://jobs/7da56f9f-51e4-4a13-8039-55127154f8d3/outputs/artifacts/paths/model/" as a shorthand to the same location
    name="run-model-example",
    description="Model created from run.",
    model_format="mlflow"
)
ml_client.models.create_or_update(run_model)

# Next Steps
- Deploy a model to an Online Inferencing endpoint - [example](/sdk/endpoints/online/sample/online-inferencing-sample.ipynb) 
- Deploy a model to a Batch Inferencing endpoint - [example](/sdk/endpoints/batch/mnist-nonmlflow.ipynb)
